In [2]:
import pandas as pd
from scipy.stats.stats import pearsonr 
import  numpy as np
from sklearn.metrics import log_loss

In [3]:
data_dir = '../data/01_raw'
train_targets_scored = pd.read_csv(data_dir+'/train_targets_scored.csv')
target_scored_cols = [c for c in train_targets_scored.columns if c != 'sig_id']

In [4]:
train1 = pd.read_csv("model_1_oof.csv")
train1b = pd.read_csv("model_1b_oof.csv")
train2 = pd.read_csv("model_2_oof.csv")
train3 = pd.read_csv("model_3_oof.csv")

In [49]:
ratios= [.45,.15,.05,.35]
#ratios= [1,0,0,0]

train = train3.copy()
train[target_scored_cols] =  ratios[0]*train1[target_scored_cols] \
                            + ratios[1]*train1b[target_scored_cols] \
                            + ratios[2]*train2[target_scored_cols] \
                            + ratios[3]*train3[target_scored_cols]


In [50]:
valid_results = train_targets_scored.drop(columns=target_scored_cols).merge(train[['sig_id']+target_scored_cols], on='sig_id', how='left').fillna(0)

y_true = train_targets_scored[target_scored_cols].values
y_pred = valid_results[target_scored_cols].values

score = 0
for i in range(len(target_scored_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / 206
    
print("CV log_loss: ", score)

CV log_loss:  0.01560892935818651


In [51]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_true,y_pred))

0.6591974929006666


In [7]:
#best score
#CV log_loss:  0.015635969854070994 - 0.6674222144449534 ( train 3)

In [24]:
x = train1b[target_scored_cols].values
y = train2[target_scored_cols].values

In [25]:
np.mean([pearsonr(x[i],y[i])[0] for i in range(train1.shape[0])])

0.7752021586326

In [ ]:
#train1 - train3 0.82
#train1 - train1b 0.89
#train1 - train2 0.81
#train1 - train1b 0.77